<a href="https://colab.research.google.com/github/andygma567/LLM-experiments/blob/main/4_bit_Falcon_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Jul 16 12:20:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash
pip install -U bitsandbytes
pip install -U git+https://github.com/huggingface/transformers.git
pip install -U git+https://github.com/huggingface/accelerate.git
# required for Falcon
pip install -U einops xformers scipy

pip install langchain

Process is terminated.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import transformers
import torch
import warnings
# warnings.filterwarnings("ignore")

from transformers import BitsAndBytesConfig

# Load the model

I think cpu offloading with device map is only available for 8bit models and I think in general it's expected that one tries to put everything on gpu

ref:
- https://colab.research.google.com/drive/1VoYNfYDKcKRQRor98Zbf2-9VQTtGJ24k?usp=sharing
- https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu


 This is an error message that I previously had when using device_map="auto"

 Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.

I think the fix is to use a custom device map that puts everything on gpu: see this example maybe https://github.com/huggingface/accelerate/issues/362#issuecomment-1125328460 or passing in `device_map={'': 0}`


In [ ]:
HF_ckpt = "tiiuae/falcon-40b-instruct"
drive_ckpt = "tiiuae/falcon-40b-instruct"

In [ ]:
%%time
# Test using 4bit configs

tokenizer = AutoTokenizer.from_pretrained(HF_ckpt)

# Use a bnb config here:
# note that using a bnb config negates any bnb args passed in
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# load the model
model = AutoModelForCausalLM.from_pretrained(
    HF_ckpt,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    quantization_config=quantization_config,
    # device_map="auto", # I wonder if using a quantization_config requires me to put everything on gpu
)
# I still need to use a high ram instance because I often don't have enough RAM to offload shards

# When I try to use a V100 then I get this warning message
# I have a warning msg:
# UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
# for the v100 gpu

# takes me about 13min to load on runpod and falcon 40b is about the 80GB

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-40b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: ignored

In [ ]:
# check how much memory is used up
!nvidia-smi

Sun Jul 16 12:19:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load a pipeline

In [ ]:
%%time
pipeline = transformers.pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
)
generation_config = GenerationConfig(
    max_new_tokens=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

NameError: ignored

In [ ]:
%%time
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
   generation_config=generation_config
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

# A100 takes about 1min to load this example... but it does work in 4bit...
# I can definitely run experiments at this level...

NameError: ignored

# Test using LangChain Tools


In [ ]:
# LangChain stuff
import langchain
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

## Create a wrapper for Falcon

In [ ]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

class LocalFalcon(LLM):
    generation_config: GenerationConfig

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # pipeline returns a list of dictionaries (due to batching)
        # each dictionary has one key "generated_text"
        batch_output = pipeline(
            prompt,
            generation_config=self.generation_config
        )

        output_dict = batch_output[0]
        return output_dict["generated_text"]

In [ ]:
generation_config = GenerationConfig(
                      max_new_tokens=1024,
                      do_sample=True,
                      top_k=10,
                      num_return_sequences=1,
                      eos_token_id=tokenizer.eos_token_id,
                    )
wrapped_llm = LocalFalcon(generation_config=generation_config)
print(wrapped_llm)

LocalFalcon
Params: {}


In [ ]:
# test
%%time
question = "What is a Falcon?"
print(wrapped_llm(question))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1340 in time            │
│                                                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│   1339 │   │   │   │   │   code_2 = self.shell.compile(expr_val, source, 'eval')                 │
│ ❱ 1340 │   │   │   │   │   out = eval(code_2, glob, local_ns)                                    │
│   1341 │   │   │   except:                                                                       │
│   1342 │   │   │   │   self.shell.showtraceback()                                                │
│   1343 │   │   │   │   return                                                                    │
│ in <module>:2                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/llms/base.py:338 in __call__                   │
│                                                                                                  │
│   335 │   │   │   │   "`generate` instead."                                                      │
│   336 │   │   │   )                                                                              │
│   337 │   │   return (                                                                           │
│ ❱ 338 │   │   │   self.generate([prompt], stop=stop, callbacks=callbacks, **kwargs)              │
│   339 │   │   │   .generations[0][0]                                                             │
│   340 │   │   │   .text                                                                          │
│   341 │   │   )                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/llms/base.py:206 in generate                   │
│                                                                                                  │
│   203 │   │   │   │   )                                                                          │
│   204 │   │   │   except (KeyboardInterrupt, Exception) as e:                                    │
│   205 │   │   │   │   run_manager.on_llm_error(e)                                                │
│ ❱ 206 │   │   │   │   raise e                                                                    │
│   207 │   │   │   run_manager.on_llm_end(output)                                                 │
│   208 │   │   │   if run_manager:                                                                │
│   209 │   │   │   │   output.run = RunInfo(run_id=run_manager.run_id)                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/llms/base.py:198 in generate                   │
│                                                                                                  │
│   195 │   │   │   )                                                                              │
│   196 │   │   │   try:                                                                           │
│   197 │   │   │   │   output = (                                                                 │
│ ❱ 198 │   │   │   │   │   self._generate(                                                        │
│   199 │   │   │   │   │   │   prompts, stop=stop, run_manager=run_manager, **kwargs              │
│   200 │   │   │   │   │   )                                

## Test a Python Agent

References: https://learn.deeplearning.ai/langchain/lesson/7/agents

In [ ]:
# https://python.langchain.com/en/latest/modules/agents/toolkits/examples/python.html
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

In [ ]:
agent_executor = create_python_agent(
    llm=wrapped_llm,
    tool=PythonREPLTool(),
    verbose=True,
    handle_parsing_errors=True,
)

# setting handle_parsing_errors="Check your output and make sure it conforms!", # I think this reruns the chain with this prompt?

In [ ]:
print(agent_executor.agent.llm_chain.prompt)

input_variables=['input', 'agent_scratchpad'] output_parser=None partial_variables={} template='You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n\n\nPython_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Python_REPL]\nAction Input: the input to the a

In [ ]:
print(agent_executor.agent.llm_chain.prompt.template)

You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.


Python_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Python_REPL]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times

In [ ]:
%%time
print(pipeline("What is 17 times 10?", max_new_tokens=200)[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


What is 17 times 10?
17 times 10 is equal to 170.
CPU times: user 2.4 s, sys: 0 ns, total: 2.4 s
Wall time: 2.59 s


In [ ]:
%%time
agent_executor.run("What is 17 times 10?")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1327 in time            │
│                                                                                                  │
│   1324 │   │   if mode=='eval':                                                                  │
│   1325 │   │   │   st = clock2()                                                                 │
│   1326 │   │   │   try:                                                                          │
│ ❱ 1327 │   │   │   │   out = eval(code, glob, local_ns)                                          │
│   1328 │   │   │   except:                                                                       │
│   1329 │   │   │   │   self.shell.showtraceback()                                                │
│   1330 │   │   │   │   return                                                                    │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)              

In [ ]:
%%time
agent_executor.run("Write a Python script that prints 'Hello, world!'")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1327 in time            │
│                                                                                                  │
│   1324 │   │   if mode=='eval':                                                                  │
│   1325 │   │   │   st = clock2()                                                                 │
│   1326 │   │   │   try:                                                                          │
│ ❱ 1327 │   │   │   │   out = eval(code, glob, local_ns)                                          │
│   1328 │   │   │   except:                                                                       │
│   1329 │   │   │   │   self.shell.showtraceback()                                                │
│   1330 │   │   │   │   return                                                                    │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)              

In [ ]:
%%time
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]
agent_executor.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 9>:9                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)                                                    │
│   163 │   │   │   )                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/agents/agent.py:987 in _call                   │
│                                                                                                  │
│    984 │   │   start_time = time.time()                                                          │
│    985 │   │   # We now enter the agent loop (until it returns something).                       │
│    986 │   │   while self._should_continue(iterations, time_elapsed):                            │
│ ❱  987 │   │   │   next_step_output = self._take_next_step(                                      │
│    988 │   │   │   │   name_to_tool_map,                   

In [ ]:
langchain.debug=True
agent_executor.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")
langchain.debug=False

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:LocalFalcon] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)                                                    │
│   163 │   │   │   )                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/agents/agent.py:987 in _call                   │
│                                                                                                  │
│    984 │   │   start_time = time.time()                                                          │
│    985 │   │   # We now enter the agent loop (until it returns something).                       │
│    986 │   │   while self._should_continue(iterations, time_elapsed):                            │
│ ❱  987 │   │   │   next_step_output = self._take_next_step(                                      │
│    988 │   │   │   │   name_to_tool_map,                   

In [ ]:
langchain.debug=False

## Test a few-shot prompt for using Python



In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
examples = [
  {
    "question": "Find out how much 2 plus 2 is.",
    "answer":
"""
Thought: I must use the Python shell to calculate 2 + 2
Action: Python REPL
Action Input:
2 + 2
Observation: 4

Thought: I now know the answer
Final Answer: 4
"""
  },
  {
    "question": "You have a variable age in your scope. If it's greater or equal than 21, say OK. Else, say Nay.",
    "answer":
"""
Thought: I should write an if/else block in the Python shell.
Action: Python REPL
Action Input:
if age >= 21:
    print("OK")  # this line has four spaces at the beginning
else:
    print("Nay")  # this line has four spaces at the beginning

Observation: OK
Thought: I now know the answer
Final Answer: I have executed the task successfully.
"""
  },
  {
    "question": "Write and execute a script that sleeps for 2 seconds and prints 'Hello, World'",
    "answer":
"""
Thought: I should import the sleep function.
Action: Python REPL
Action Input:
from time import sleep
Observation:

Thought: I should call the sleep function passing 2 as parameter
Action: Python REPL
Action Input:
from time import sleep
sleep(2)
Observation:

Thought: I should use the 'print' function to print 'Hello, World'
Action: Python REPL
Action Input:
from time import sleep
sleep(2)
print('Hello, World')
Observation: Hello, World

Thought: I now finished the script
Final Answer: I executed the following script successfully:

from time import sleep
sleep(2)
print('Hello, World')
"""
  },
]

In [ ]:
template_string="""Example:
Question: {question}
{answer}
"""

In [ ]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template=template_string)
print(example_prompt.format(**examples[0]))

Example:
Question: Find out how much 2 plus 2 is.

Thought: I must use the Python shell to calculate 2 + 2
Action: Python REPL
Action Input:
2 + 2
Observation: 4

Thought: I now know the answer
Final Answer: 4




In [ ]:
suffix_string="""
Additional Hints:
1. If an error thrown along the way, try to understand what happened and retry with a new code version that fixes the error.
2. DO NOT IGNORE ERRORS.
3. If an object does not have an attribute, call dir(object) to debug it.
4. SUPER IMPORTANT: ALWAYS respect the indentation in Python. Loops demand an idendentation. For example:

for i in range(10):
    print(i)  # this line has four spaces at the beginning

Same for ifs:

if True:
    print("hello")  # this line has four spaces at the beginning

An error be thrown because of the indentation, something like...  "expected an indented block after 'for' statement on line..."

To fix, make sure to indent the lines!

5. Do not use \ in variable names, otherwise you'll see the syntax error "unexpected character after line continuation character..."
6. If the variable is not defined, use vars() to see the defined variables.
7. Do not repeat the same statement twice without a new reason.
8. NEVER print the HTML directly.

Now begin for real!

Question: {input}
"""

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix=suffix_string,
    input_variables=["input"]
)

In [ ]:
print(prompt.format(input="Write a Python script that prints 'Hello, world!'"))

Example:
Question: Find out how much 2 plus 2 is.

Thought: I must use the Python shell to calculate 2 + 2
Action: Python REPL
Action Input:
2 + 2
Observation: 4

Thought: I now know the answer
Final Answer: 4



Example:
Question: You have a variable age in your scope. If it's greater or equal than 21, say OK. Else, say Nay.

Thought: I should write an if/else block in the Python shell.
Action: Python REPL
Action Input:
if age >= 21:
    print("OK")  # this line has four spaces at the beginning
else:
    print("Nay")  # this line has four spaces at the beginning

Observation: OK
Thought: I now know the answer
Final Answer: I have executed the task successfully.



Example:
Question: Write and execute a script that sleeps for 2 seconds and prints 'Hello, World'

Thought: I should import the sleep function.
Action: Python REPL
Action Input:
from time import sleep
Observation:

Thought: I should call the sleep function passing 2 as parameter
Action: Python REPL
Action Input:
sleep(2)
Obs

In [ ]:
%%time
question1 = "Write a Python script that prints 'Hello, world!'"
agent_executor.run(prompt.format(input=question1))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1340 in time            │
│                                                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│   1339 │   │   │   │   │   code_2 = self.shell.compile(expr_val, source, 'eval')                 │
│ ❱ 1340 │   │   │   │   │   out = eval(code_2, glob, local_ns)                                    │
│   1341 │   │   │   except:                                                                       │
│   1342 │   │   │   │   self.shell.showtraceback()                                                │
│   1343 │   │   │   │   return                                                                    │
│ in <module>:2                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)              

In [ ]:
# This cell takes way too long to run
# I should put a limit on how many iterations it's allowed to do

%%time
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]
question2 = f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}"""
agent_executor.run(prompt.format(input=question2))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1340 in time            │
│                                                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│   1339 │   │   │   │   │   code_2 = self.shell.compile(expr_val, source, 'eval')                 │
│ ❱ 1340 │   │   │   │   │   out = eval(code_2, glob, local_ns)                                    │
│   1341 │   │   │   except:                                                                       │
│   1342 │   │   │   │   self.shell.showtraceback()                                                │
│   1343 │   │   │   │   return                                                                    │
│ in <module>:12                                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)              